In [ ]:
# %pip install -q "pydantic>=2" langchain langchain-openai langchain-community

In [2]:
from typing import List
from pydantic import BaseModel, Field

from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI


try:
    from langchain_core.output_parsers import PydanticOutputParser
except ImportError:
    
    from langchain.output_parsers import PydanticOutputParser  # fallback


class StudentInfo(BaseModel):
    name: str = Field(description="학생 이름")
    age: int = Field(description="나이(정수)")
    major: str = Field(description="전공")
    hobbies: List[str] = Field(description="취미 리스트")
    goal: str = Field(description="장래 목표 또는 계획")


def build_llm(model_name: str = "gpt-4o-mini", temperature: float = 0.0):
    return ChatOpenAI(model=model_name, temperature=temperature)


In [3]:
def parse_student_bio(bio: str, llm=None) -> StudentInfo:
    """
    자유 형식 자기소개(bio)에서
    name(str), age(int), major(str), hobbies(list[str]), goal(str)을 추출해 StudentInfo로 반환.
    """
    llm = llm or build_llm()

    
    parser = PydanticOutputParser(pydantic_object=StudentInfo)
    fmt = parser.get_format_instructions()  
    
    prompt = PromptTemplate(
        template=(
            """
            아래 자유 형식 자기소개에서 학생 정보를 구조화해라.
            필드: name(str), age(int), major(str), hobbies(list[str]), goal(str)
            {format_instructions}

            자기소개: {bio}
            """
        ),
        input_variables=["bio"],
        partial_variables={"format_instructions": fmt},
    )

    chain = prompt | llm | parser
    return chain.invoke({"bio": bio})


In [4]:
sample = (
    "안녕하세요! 저는 김민수이고 22살입니다. 컴퓨터공학을 전공하고 있어요. "
    "취미로는 게임하기, 영화보기, 코딩을 좋아합니다. 앞으로 훌륭한 개발자가 되는 것이 목표입니다."
)

result = parse_student_bio(sample)
print(result.model_dump())  


{'name': '김민수', 'age': 22, 'major': '컴퓨터공학', 'hobbies': ['게임하기', '영화보기', '코딩'], 'goal': '훌륭한 개발자가 되는 것'}
